<a href="https://colab.research.google.com/github/AsadbekTuronov/data_science_and_AI/blob/main/amaliyot_avia_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'avia-sales:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5854619%2F9597431%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241012%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241012T014103Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1464af7d6c23ee14e2ced66f45623ba8820a8f906607a02f3b9e0c36c2534534d88da058cd665f4404eab37b34f3e5339cad32450b4380cbf8d182b1415c47f7f1653329ac1b707239a4030afc276fe965606811cd3ea3daec999461e339033aef20a99357094c74ff368f68440c45135cf6bc81f25eb9d06e2afb34039fa5c834db1ebe661b45ba05a5c07b6e8a14848df932b5336f2e59a902ddabcfc108a37c63dadd6ac5ce64227b5343d1ff14d095deba2d20023d54c20885b02dca25e1d50f92d04bb66179615ecdbf0c689a69250beb16da6e03a0350fb868b65cb5ca37b046b756d6bce3528ecec7597beae287b6e46bd61ae659765e72e00c51b700'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**show data definition**

In [ ]:
df = pd.read_csv("/kaggle/input/avia-sales/train_data.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

**Data preproccessing**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder

In [ ]:
def drop_NaN_rows(x):
    return x.dropna()

In [ ]:
column_transformer=ColumnTransformer(transformers=[
    ("onehotencoder", OneHotEncoder(drop='first'), ["departure_time", "stops", "arrival_time", "class"],)
], remainder='passthrough')

In [ ]:
pipe1 = Pipeline(steps=[
    ("drop NaN rows", FunctionTransformer(drop_NaN_rows)),
    ("column transformer", column_transformer)
])

In [ ]:
a = pipe1.fit_transform(df)
ocl = pipe1.named_steps['column transformer'].named_transformers_['onehotencoder'].get_feature_names_out(["departure_time", "stops", "arrival_time", "class"])
a1l = list(ocl)+[i for i in df.columns if i not in ["departure_time", "stops", "arrival_time", "class"]]
t_data = pd.DataFrame(a, columns=a1l)

In [ ]:
t_data.sample(5)

In [ ]:
for i in ["id", "airline", "flight", "source_city", "destination_city"]:
    t_data.drop(i, axis=1, inplace=True)

In [ ]:
t_data.corrwith(t_data["price"])

**modelling**

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, PoissonRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, BaggingRegressor, StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
x, y = t_data.drop("price", axis=1), t_data["price"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
linearmodel = LinearRegression()
linearmodel.fit(x_train, y_train)

In [ ]:
linearpred = linearmodel.predict(x_test)

In [ ]:
ridgeregressionmodel = Ridge(1.0)
ridgeregressionmodel.fit(x_train, y_train)

In [ ]:
ridgeregressionpred = ridgeregressionmodel.predict(x_test)

In [ ]:
poissonregressormodel = PoissonRegressor()
poissonregressormodel.fit(x_train, y_train)

In [ ]:
poissonregressorpred = poissonregressormodel.predict(x_test)

In [ ]:
adaboostregressormodel = AdaBoostRegressor()
adaboostregressormodel.fit(x_train, y_train)

In [ ]:
adaboostregressorpred = adaboostregressormodel.predict(x_test)

In [ ]:
randomforestregressormodel = RandomForestRegressor()
randomforestregressormodel.fit(x_train, y_train)

In [ ]:
randomforestregressorpred = randomforestregressormodel.predict(x_test)

In [ ]:
baggingregressormodel = BaggingRegressor()
baggingregressormodel.fit(x_train, y_train)

In [ ]:
baggingregressorpred = baggingregressormodel.predict(x_test)

In [ ]:
base_regs = [
    ("linear", LinearRegression()),
    ("ridge", Ridge()),
    ("PoissonRegressor", PoissonRegressor()),
    ("AdaBoostRegressor", AdaBoostRegressor()),
    ("BaggingRegressor", BaggingRegressor())
]

In [ ]:
meta_reg = RandomForestRegressor()

In [ ]:
stacking_regressor = StackingRegressor(estimators=base_regs, final_estimator=meta_reg)

In [ ]:
stacking_regressor.fit(x_train, y_train)

In [ ]:
stackingregressorpred = stacking_regressor.predict(x_test)

In [ ]:
def show_mae_mse(model_name, model):
    print(model_name, " "*(25-len(model_name)), "mean_absolute_error:", mean_absolute_error(y_test, model), " "*(20-len(str(mean_absolute_error(y_test, model)))), "mean_squared_error:", mean_squared_error(y_test, model))

In [ ]:
show_mae_mse("LinearRegression", linearpred)
show_mae_mse("Ridge", ridgeregressionpred)
show_mae_mse("PoissonRegressor", poissonregressorpred)
show_mae_mse("AdaBoostRegressor", adaboostregressorpred)
show_mae_mse("BaggingRegressor", baggingregressorpred)
show_mae_mse("RandomForestRegressor", randomforestregressorpred)
show_mae_mse("StackingRegressor", stackingregressorpred)